# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

In [2]:
data_train = pd.read_csv('data/train.csv')
data_test = pd.read_csv('data/test.csv')
data = pd.concat([data_train,data_test])
sparse_features = ['C' + str(i) for i in range(1, 16)]
dense_features = ['I' + str(i) for i in range(1, 3)]
data_train = []
data_test= []

(20317220, 19)


In [3]:
data[sparse_features] = data[sparse_features].fillna(-1, )
data[dense_features] = data[dense_features].fillna(0, )

In [4]:
target = ['Label']
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [5]:
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                          for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(
    linear_feature_columns + dnn_feature_columns)

train, test = train_test_split(data, test_size=20317220, shuffle=False)

train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

In [6]:
data=[]
y=train[target].values
train=[]
test=[]

In [8]:
model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
               task='binary',
               l2_reg_embedding=1e-4, device=device)
model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy"], )

In [22]:
model.fit(train_model_input, y,
          batch_size=2499146, epochs=11,  verbose=2)

cpu
Train on 29989752 samples, validate on 0 samples, 12 steps per epoch
Epoch 1/2
1695s - loss:  0.0150 - binary_crossentropy:  0.0150
Epoch 2/2
1675s - loss:  0.0145 - binary_crossentropy:  0.0144


In [23]:
pred = model.predict(test_model_input)

In [24]:
with open("UpDeepFM.csv", 'w') as f:
    f.write("Id,Click\n")
    for i,j in enumerate(pred):
        f.write(str(i+1)+","+str(j[0])+"\n")

In [45]:
with open("deepCTR_5ep_gpu_12ep.csv", 'r') as f:
    for i,j in enumerate(f.readlines()):
        if(i==0):
            continue
        line = j.split(",")
        s=float(line[1])
        deep12.append(s)

In [34]:
with open("submission.csv", 'r') as f:
    for i,j in enumerate(f.readlines()):
        if(i==0):
            continue
        line = j.split(",")
        s=float(line[1])
        y_pred.append(s)
        

In [46]:
wampal = []
with open("wampal.csv", 'r') as f:
    for i,j in enumerate(f.readlines()):
        if(i==0):
            continue
        line = j.split(",")
        s=float(line[1])
        wampal.append(s)

In [49]:
deep5 = []
with open("wampal.csv", 'r') as f:
    for i,j in enumerate(f.readlines()):
        if(i==0):
            continue
        line = j.split(",")
        s=float(line[1])
        deep5.append(s)

In [51]:
with open("VW_ffm_deep11.csv", 'w') as f:
    f.write("Id,Click\n")
    for i in range(len(y_pred)):
        res=0.2*y_pred[i]+0.6*wampal[i]+0.2*pred[i][0]
        f.write(str(i+1)+","+str(res)+'\n')

In [48]:
with open("All.csv", 'w') as f:
    f.write("Id,Click\n")
    for i in range(len(y_pred)):
        res=0.2*ffm[i]+0.7*wampal[i]+0.1*deep11[i]
        f.write(str(i+1)+","+str(res)+'\n')

In [50]:
with open("VW+deep5.csv", 'w') as f:
    f.write("Id,Click\n")
    for i in range(len(y_pred)):
        res=0.8*wampal[i]+0.2*deep5[i]
        f.write(str(i+1)+","+str(res)+'\n')

# Analysis/Modeling
Do work here

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here